# Video runner

same as live runner but for getting data from capture

In [1]:
import ifxdaq
import processing
import numpy as np
#print(ifxdaq.__version__)
from ifxdaq.sensor.radar_ifx import RadarIfxAvian
import matplotlib.pyplot as plot
from matplotlib.animation import FuncAnimation
import torch

from dataset import preprocess_input

device = "cuda"
config_file = "radar_configs/RadarIfxBGT60.json"

In [6]:
import cv2
import sys

from models import *
f = 0
model = SpaghettiModel().to(device)
model.load_state_dict(torch.load('models/spaghettiAlPestoETonno.pth'))
model.eval()

file_name = f'../../recordings/recording_2022_05_26_17_33_25/RadarIfxAvian_00/radar.npy'
data = np.load(file_name)

range_doppler_map = processing.processing_rangeDopplerData(data)
preprocessed = preprocess_input(range_doppler_map)

history = torch.zeros((10)).to(device)

for frame in preprocessed:
    # Loop through the frames coming from the radar
    input = torch.from_numpy(frame).type(torch.FloatTensor).to(device=device)
    input = torch.unsqueeze(input, 0)

    no = torch.argmax(model(input))
    history = torch.cat([torch.unsqueeze(no,0), history[:9]])
    sys.stdout.write(f"\rDetecting {torch.median(history).item()} people")
    sys.stdout.flush()
    data = preprocess_input([frame])


    img = np.concatenate((data[0,0,:,:],data[0,1,:,:],data[0,2,:,:] ))
    
    # scale the image
    scale_percent = 400 # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)

    img = cv2.resize(img, dim)

    text = np.zeros(shape=(256, 256,3), dtype=np.int16)
    cv2.putText(text, text=f"{no}", org=(30, 250), fontFace=cv2.FONT_HERSHEY_TRIPLEX, 
        fontScale=8, color=(255,0,0),thickness=3)
    
    cv2.imshow("img",img)
    # cv2.imshow("text",text)

    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    

Detecting 2.0 people